## Описание проекта

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [ ]:
# импорт библиотек

import pandas as pd
import sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
# <чтение файла с данными с сохранением в data_full>

data = pd.read_csv('/datasets/insurance.csv')

In [ ]:
# изучаю датафрейм

print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [ ]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [ ]:
data["Пол"].value_counts()

0    2505
1    2495
Name: Пол, dtype: int64

**Вывод:**  
Данные состроят из 5000 строк  
Признаки: Пол, Возраст, Зарплата, Члены, семьи  
Целевой признак: количество страховых выплат  
Данные в предобработке не нуждаются  
Данные разделены по полу почти в равном колличестве


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

**Ответ:** Не изменится.

**Обоснование:**  

Признаки умножают на обратимую матрицу $P$ : 

$$
w_1 = ((XP)^T XP)^{-1} (XP)^T y
$$


Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке

$$
w_1 = (P^T X^T XP)^{-1} P^T X^T  y
$$

Используя формулу
$$
(A * B)^{-1} = B^{-1} * A^{-1}  
$$  

где $A$ и $B$ квадратные  

$P$, $P^T$ и $X^T X$ квадратные  


$$
w_1= P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T  y
$$


Умножение матрицы на обратную матрицу равно единичной матрице  
$$
w_1= P^{-1} (X^T X)^{-1} I X^T  y
$$

Умножение любой матрицы на единичную равно этой самой матрице.
$$
w_1= P^{-1} (X^T X)^{-1} X^T  y
$$

$$
a_1 = XPw_1
$$  

$$
a_1 = X P P^{-1} (X^T X)^{-1} X^T  y
$$  

$$
a_1 = X (X^T X)^{-1} X^T  y
$$  

$$
a_1 = Xw
$$

**Вывод:**  
Преобразовав формулы мы выяснили что при умножении признаков на обратимую матрицу качество не меняется

## Алгоритм преобразования

**Алгоритм**  
Необходимо умножить матрицу признаков на матрицу шифрования так как при умножении количество столбцов матрицы A должно быть равно количеству строк матрицы, полученная матрица будет иметь количество строк матрицы A и количество столбцов матрицы B. Поэтому размер матрицы щифрования на которую умножают равен 4х4

Матрицу шифрования сгенерируем с помощью np.random.normal()

**Обоснование**


In [ ]:
#разобьем data на признаки и целевой признак
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [ ]:
# Создадим обратимую квадратную матрицу со случайными числами размерностью столбцов features
state = np.random.RandomState(12345)
matrix_cr=np.random.normal(size=(features.shape[1],features.shape[1]))

In [ ]:
# зашифруем даннные путем умножения матриц
features_cr=features @ matrix_cr
features_cr

,0,1,2,3
0,30993.856535,-24217.279968,21561.850518,4641.038069
1,23743.752600,-18554.023274,16509.182987,3572.734854
2,13121.220017,-10253.719555,9122.178171,1978.932944
3,26059.386000,-20358.323444,18134.741121,3883.972971
4,16308.233196,-12744.293695,11342.566236,2450.763677
...,...,...,...,...
4995,22308.391863,-17429.970674,15518.099368,3337.475325
4996,32745.487213,-25582.472300,22785.887189,4890.267851
4997,21184.464100,-16550.618169,14739.915935,3161.030126
4998,20433.237499,-15966.273739,14213.781144,3053.477467


**Вывод**  
После умножения размер матрицы признаков и кодированных признаков совпадает

## Проверка алгоритма

### Качество линейной регресии на нешифрованных данных

In [ ]:
model = LinearRegression()

model.fit(features, target)

predictions = model.predict(features)

print('R2:', r2_score(target,predictions))

R2: 0.42494550286668


### Качество линейной регресии c шифрованием данных

In [ ]:
model = LinearRegression()

model.fit(features_cr, target)

predictions = model.predict(features_cr)

print('R2:', r2_score(target,predictions))

R2: 0.4249455028666821


**Вывод:**  
Качество моделей с шифрованием при помощи умножения на обратимую матрицу со случайными числами не изменяется  
Небольшое расхождение предположу связанно с тем, что операции с float накапливают небольшую погрешность.